# Revised case normalization for Hirslanden Linde 2018





In [1]:
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')

import os

from py.global_configs import *
from py.normalize import normalize

from src.service import bfs_cases_db_service as bfs_db

from src.utils.dataframe_utils import remove_duplicated_chops

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:

file_info = FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx'),
         'Hirslanden Linde',
         '2018',
         ['Änderungen_LI_2018'])
print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx', hospital_name_db='Hirslanden Linde', year='2018', sheets=['Änderungen_LI_2018'])


In [3]:
cols_to_rename = dict(COLUMNS_TO_RENAME)
# Replace 'admno' with 'fall nummer'
cols_to_rename.pop('admno')
cols_to_rename['fall nummer'] = CASE_ID_COL

df_revised_case_d2d = normalize(file_info, 0)

2022-10-18 12:13:24.192 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 0: discarded ICDs after validation {''}
2022-10-18 12:13:24.192 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 2: discarded ICDs after validation {''}
2022-10-18 12:13:24.193 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 3: discarded ICDs after validation {''}
2022-10-18 12:13:24.193 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 4: discarded ICDs after validation {''}
2022-10-18 12:13:24.194 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 7: discarded ICDs after validation {''}
2022-10-18 12:13:24.194 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 9: discarded ICDs after validation {''}
2022-10-18 12:13:24.196 | DEBUG    | src.utils.dataframe_utils:_validate_chop_codes:94 - row 0: discarded duplicated and invalid CHOP entries after validation {'990410::20180124'}
2022-10-18 12:13:24.197 | DE

Read 10 cases for Hirslanden Linde 2018
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
old_pd              string
new_pd              string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


In [4]:
df_revised_case_d2d

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,715177,33100584B601FADF,M,90,11,3,G18B,C182,C182,M200,[D62],[],[],[]
1,716197,DEE16D56D71DD96A,W,72,14,3,I09C,M4805,M4805,M200,[N184],[N183],[],[]
2,721128,34645F6C19043F5B,M,71,14,3,G21B,K565,K565,M200,[G819],[],[],[]
3,721977,0098EFC426FD8F26,W,84,11,3,I46C,S7201,S7201,M200,[T840],[],[],[]
4,725531,EBF3D9B44B52E53F,M,80,2,3,F59E,I7022,I7022,M100,[G2090],[],[],[]
5,727730,1FECD162D99A5290,W,91,9,2,E77F,J181,J181,M100,[N183],[N189],[],[]
6,727952,5415F800B9F4D0D6,M,88,4,3,F62C,I5019,I5019,M100,[I420],[I517],[],[]
7,728588,F70905DAE9903086,W,84,10,3,E77C,A481,A481,M100,[R64],[],[890A32],[]
8,731638,43A277B8C7954BC7,W,76,7,2,I29C,M751,M751,M200,[J9610],[J9699],[],[]
9,737831,8FC3EAD412C59FE3,W,73,5,2,E77F,J181,J181,M100,[G8253],[],[],[]


# Match revised DtoD data with the bfs data from the database


In [5]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['715177', '716197', '721128', '721977', '725531', '727730',
       '727952', '728588', '731638', '737831'], dtype=object)

In [6]:
revised_case_id_stripped = [case_id.lstrip('0') for case_id in revised_case_id]

### Check if there are duplicated case_ids in the revised cases after stripping the 0 in front, if yes, delete duplicated cases

In [7]:

revised_case_id_stripped_set = set(revised_case_id_stripped)
if len(revised_case_id_stripped) > len(revised_case_id_stripped_set):
    print('There are duplicated case_ids in the revised cases.' )
else:
    print('There are no duplicated case_ids in the revised cases.' )

There are no duplicated case_ids in the revised cases.


In [8]:
# match to the database and retrieve data from database using revised
revised_case_db = bfs_db.get_bfs_cases_by_ids_no_pad0(list(revised_case_id_stripped_set))
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl
0,2.967,135211,7,716197,DEE16D56D71DD96A,72,0,W,14,4,0,0,0,2018-02-14,01,2018-02-28,00,I09C,I09,3
1,0.507,329457,13,900716197,108DD197AF1A4761,86,0,W,4,16,0,0,0,2019-10-30,01,2019-11-03,00,L63E,L63,2
0,1.644,132931,7,721977,0098EFC426FD8F26,84,0,W,11,4,0,0,0,2018-02-24,01,2018-03-07,00,I46C,I46,3
0,1.132,134378,7,728588,F70905DAE9903086,84,0,W,10,3,0,0,0,2018-04-13,01,2018-04-23,00,E77C,E77,3
0,0.769,137478,7,731638,43A277B8C7954BC7,76,0,W,7,4,0,0,0,2018-07-12,01,2018-07-19,00,I29C,I29,2


In [9]:
# create a new column for case_id_stripped with '0' for joining to the revised cases
revised_case_db['case_id_stripped']=  revised_case_db['case_id'].apply(lambda x: x.lstrip('0')) 
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,...,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl,case_id_stripped
0,2.967,135211,7,716197,DEE16D56D71DD96A,72,0,W,14,4,...,0,0,2018-02-14,01,2018-02-28,00,I09C,I09,3,716197
1,0.507,329457,13,900716197,108DD197AF1A4761,86,0,W,4,16,...,0,0,2019-10-30,01,2019-11-03,00,L63E,L63,2,900716197
0,1.644,132931,7,721977,0098EFC426FD8F26,84,0,W,11,4,...,0,0,2018-02-24,01,2018-03-07,00,I46C,I46,3,721977
0,1.132,134378,7,728588,F70905DAE9903086,84,0,W,10,3,...,0,0,2018-04-13,01,2018-04-23,00,E77C,E77,3,728588
0,0.769,137478,7,731638,43A277B8C7954BC7,76,0,W,7,4,...,0,0,2018-07-12,01,2018-07-19,00,I29C,I29,2,731638


In [15]:
revised_case_db

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,...,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl,case_id_stripped
0,2.967,135211,7,716197,DEE16D56D71DD96A,72,0,W,14,4,...,0,0,2018-02-14,01,2018-02-28,00,I09C,I09,3,716197
1,0.507,329457,13,900716197,108DD197AF1A4761,86,0,W,4,16,...,0,0,2019-10-30,01,2019-11-03,00,L63E,L63,2,900716197
0,1.644,132931,7,721977,0098EFC426FD8F26,84,0,W,11,4,...,0,0,2018-02-24,01,2018-03-07,00,I46C,I46,3,721977
0,1.132,134378,7,728588,F70905DAE9903086,84,0,W,10,3,...,0,0,2018-04-13,01,2018-04-23,00,E77C,E77,3,728588
0,0.769,137478,7,731638,43A277B8C7954BC7,76,0,W,7,4,...,0,0,2018-07-12,01,2018-07-19,00,I29C,I29,2,731638
0,0.636,133862,7,727730,1FECD162D99A5290,91,0,W,9,3,...,0,0,2018-04-07,01,2018-04-16,00,E77F,E77,2,727730
0,0.636,133953,7,737831,8FC3EAD412C59FE3,73,0,W,5,3,...,0,0,2018-06-20,01,2018-06-25,00,E77F,E77,2,737831
0,1.318,132057,7,721128,34645F6C19043F5B,71,0,M,14,4,...,0,0,2018-02-19,01,2018-03-05,00,G21B,G21,3,721128
0,0.858,133437,7,727952,5415F800B9F4D0D6,88,0,M,4,3,...,0,0,2018-04-10,01,2018-04-14,00,F62C,F62,3,727952
0,1.781,137694,7,715177,33100584B601FADF,90,0,M,11,4,...,0,0,2018-01-22,01,2018-02-02,00,G18B,G18,3,715177


In [10]:
# create a new column for case_id_stripped with '0' for joining to the revised cases
revised_case_db['case_id_stripped']=  revised_case_db['case_id'].apply(lambda x: x.lstrip('0')) 
revised_case_db.head()
print(revised_case_db.shape)

(11, 21)


In [11]:
# create a new column for case_id_stripped with '0' for joining to cases from database

df_revised_case_d2d['case_id_stripped']=  df_revised_case_d2d['case_id'].apply(lambda x: x.lstrip('0')) 
df_revised_case_d2d.head()
print(df_revised_case_d2d.shape)

(10, 15)


In [12]:
# delete duplicated cases in the revised cases

df_revised_case_d2d['case_id_stripped'].apply(tuple) # make sure the drop_duplicates works

df_revised_case_d2d.drop_duplicates(subset=['case_id_stripped'], keep='last', inplace=True)
assert len(revised_case_id_stripped) == df_revised_case_d2d.shape[0] # make sure all cases are deleted


# Forming the normalized dataframe: Merge the selected revised case data with the matched from bfs data
  If there are multiple cases were found based on case_id, patient_id will be used to match once more  

In [50]:
revised_case_db_val_subset = revised_case_db[VALIDATION_COLS]

row_example_db = revised_case_db_val_subset[revised_case_db_val_subset['case_id']=='716197']

In [51]:
revised_case_db_val_subset

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg
0,716197,DEE16D56D71DD96A,W,72,14,3,I09C
1,900716197,108DD197AF1A4761,W,86,4,2,L63E
0,721977,0098EFC426FD8F26,W,84,11,3,I46C
0,728588,F70905DAE9903086,W,84,10,3,E77C
0,731638,43A277B8C7954BC7,W,76,7,2,I29C
0,727730,1FECD162D99A5290,W,91,9,2,E77F
0,737831,8FC3EAD412C59FE3,W,73,5,2,E77F
0,721128,34645F6C19043F5B,M,71,14,3,G21B
0,727952,5415F800B9F4D0D6,M,88,4,3,F62C
0,715177,33100584B601FADF,M,90,11,3,G18B


In [52]:
df_revised_case_d2d_val_subset = df_revised_case_d2d[VALIDATION_COLS]
row_example_db_dtod = revised_case_db_val_subset[revised_case_db_val_subset['case_id']=='716197']

In [77]:
len(row_example_db[VALIDATION_COLS].values[0] == row_example_db_dtod[VALIDATION_COLS].values[0])

7

In [78]:
def validation_function(df, df_db, val_col, case_id_stripped_val=list()):
    df_val_subset = df[val_col]
    df_db_val_subset = df_db[val_col]
    for case_id_stripped in df['case_id_stripped'].values:
        df_val_subset_row = df_val_subset[df_val_subset['case_id_stripped']==case_id_stripped]
        df_db_val_subset_row = df_db_val_subset[df_db_val_subset['case_id_stripped']==case_id_stripped]
        if len(df_val_subset_row[val_col].values[0] == df_db_val_subset_row[val_col].values[0]) == len(val_col):
            case_id_stripped_val.append(case_id_stripped)
    return case_id_stripped_val

In [58]:
VALIDATION_COLS

['case_id',
 'patient_id',
 'gender',
 'age_years',
 'duration_of_stay',
 'pccl',
 'drg']

In [79]:
val_col = ['case_id_stripped', 'gender',  'age_years',  'duration_of_stay',  'pccl',  'drg']

In [81]:
case_id_stripped_val = validation_function(df_revised_case_d2d, revised_case_db, val_col)

In [17]:
# Check if there are multiple cases were found based on case_id, patient_id will be used to match once more





if revised_case_db.shape[0] <= len(revised_case_id):
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_d2d_subset.merge(revised_case_db, how ='right', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_id = revised_case_normalized.shape[0]
else:
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_db.merge(revised_case_d2d_subset, how ='left', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_patient_id = revised_case_normalized.shape[0]
print(revised_case_normalized.shape)
revised_case_normalized.head()

(11, 29)


,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id_x,age_years,age_days,gender,duration_of_stay,clinic_id,...,pccl,case_id_stripped,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops,patient_id_y
0,2.967,135211,7,716197,DEE16D56D71DD96A,72,0,W,14,4,...,3,716197,M4805,M4805,M200,[N184],[N183],[],[],DEE16D56D71DD96A
1,0.507,329457,13,900716197,108DD197AF1A4761,86,0,W,4,16,...,2,900716197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.644,132931,7,721977,0098EFC426FD8F26,84,0,W,11,4,...,3,721977,S7201,S7201,M200,[T840],[],[],[],0098EFC426FD8F26
3,1.132,134378,7,728588,F70905DAE9903086,84,0,W,10,3,...,3,728588,A481,A481,M100,[R64],[],[890A32],[],F70905DAE9903086
4,0.769,137478,7,731638,43A277B8C7954BC7,76,0,W,7,4,...,2,731638,M751,M751,M200,[J9610],[J9699],[],[],43A277B8C7954BC7


In [14]:
# Choose column to export to a csv file
df_revised_case_normalized = revised_case_normalized[COLS_TO_OUTPUT]
assert len(df_revised_case_normalized.columns) == len(COLS_TO_OUTPUT)
df_revised_case_normalized.head()

KeyError: "['patient_id'] not in index"

# Export the normalized revised cases with the neccessary columns to a csv file


In [ ]:

hospital = file_info.hospital_name_db.replace(' ', '_') # replace spaces with '_' for the file name
hospital_year = f'{hospital}_{file_info.year}'

output_folder = os.path.join(ROOT_DIR, 'normalized_revision_data/')

if not os.path.exists(output_folder):
    os.mkdir(output_folder)

output_path = os.path.join(ROOT_DIR, 'normalized_revision_data/') + hospital_year + '.csv'

df_revised_case_normalized.to_csv(output_path)


# Report

A small summary of:
- how many cases could be matched in the DB, and based on which columns
- how many cases were “correctly” revised, in terms of upcoding (still need to do after the `grouper` is available for python)

In [ ]:
if revised_case_db.shape[0] <= len(revised_case_id):
    print(f'Based on the columns "case_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )
else:
    print(f'Based on the columns "case_id" and "Patient_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_patient_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )

In [ ]:
print('All the normalized revised cases are exported to here: {}'.format(output_path))